# 1. Tratamento dos Dados

Notebook com o objetivo de importar e tratar os dados referentes a cobertura de vacinação do estado Piauí, Região Nordeste e de todo o território nacional.

Os dados disponíveis são oriundos do Sistema de Avaliação do Programa de Imunizações - API, sistema este gerido pela Coordenação Geral do Programa Nacional de Imunizações - CGPNI, do Departamento de Vigilância Epidemiológica, secretaria de Vigilância em Saúde, do Ministério da Saúde, em conjunto com as Secretarias Estaduais de Saúde, suas regionais e as Secretarias Municipais de Saúde. Retirados pelo site do DataSus.

## 1.1. Descrição dos Dados
- Imuno: descrição Imunobiológico.
- Variable: período anual da cobertura vacinal (dados atualizados em: 04/09/2019).
- Value: cobertura vacinal que representam o número de doses aplicadas dividido pela população alvo.

In [1]:
# Importação de pacotes
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
# Criando variáveis com o diretório de cada arquivo
dir_piaui = '../data/raw/coberturas_vacinais_por_ano_segundo_imuno_piaui.csv'
dir_nordeste = '../data/raw/coberturas_vacinais_por_ano_segundo_imuno_regiao_nordeste.csv'
dir_brasil = '../data/raw/coberturas_vacinais_por_ano_segundo_imuno_brasil.csv'

In [3]:
# Criando a função para gerar o dataframe com o local
def create_dataframe(dir_csv, local, skip_rows):
    df = pd.read_csv(dir_csv,
                     encoding="ISO-8859-1",
                     skiprows=skip_rows,
                     sep = ";",
                     skipfooter=19, # Ignorar 12 últimas linhas
                     thousands=".", # Separador de milhares
                     decimal=",") # Separador decimal) 
    df.drop(columns=['Total'], inplace=True)
    df = df.melt(id_vars='Imuno')
    df['local'] = f'{local}'
    df['value'] = df['value'].replace('-', 0).astype('float')
    df['variable'] = df['variable'].astype('int')
    df = df.dropna()
    return df

In [4]:
# Criando o dataframe por Local Estudado
df_piaui = create_dataframe(dir_piaui,'Piaui', 4)
df_nordeste = create_dataframe(dir_nordeste,'Nordeste', 4)
df_brasil = create_dataframe(dir_brasil,'Brasil', 3)

In [5]:
df_geral = df_piaui
df_geral = df_geral.append(df_nordeste, ignore_index=True)
df_geral = df_geral.append(df_brasil, ignore_index=True)
df_geral.head()

,Imuno,variable,value,local
0,072 BCG,1994,99.20,Piaui
1,073 Hepatite B,1994,0.00,Piaui
2,074 Poliomielite,1994,53.33,Piaui
3,006 Febre Amarela,1994,0.01,Piaui
4,075 DTP,1994,65.65,Piaui


In [6]:
# Exportando Arquivo para o .csv
df_geral.to_csv('../data/processed/coberturas_vacinais_por_ano_segundo_imuno_todas_regioes.csv',
               sep=";",
               index=False)
